# 統計基礎② - 確率・分布

確率論と確率分布の理論的背景から実践的応用まで詳しく学習します。
これらは機械学習とデータサイエンスの基礎となる重要な概念です。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
from scipy import stats
from scipy.special import factorial
import warnings
import itertools
from collections import Counter

# 設定
warnings.filterwarnings('ignore')
plt.style.use('default')
np.random.seed(42)

## 1. 確率論の基礎

確率の公理とその性質について学習します。

### 1.1 確率の公理（コルモゴロフの公理）

In [ ]:
# 確率の基本性質の確認
print("確率の基本性質の検証")
print("=" * 30)

# サイコロの例で確認
# 全事象の確率の和 = 1
dice_probabilities = [1/6] * 6
total_prob = sum(dice_probabilities)
print(f"サイコロの全事象の確率の和: {total_prob}")

# 排反事象の確率の和
prob_even = sum([1/6, 1/6, 1/6])  # 2, 4, 6
prob_odd = sum([1/6, 1/6, 1/6])   # 1, 3, 5
print(f"偶数が出る確率: {prob_even:.3f}")
print(f"奇数が出る確率: {prob_odd:.3f}")
print(f"偶数 + 奇数 = {prob_even + prob_odd:.3f}")

# 余事象
prob_not_6 = 1 - 1/6
print(f"6以外が出る確率: {prob_not_6:.3f}")

### 1.2 条件付き確率とベイズの定理

In [ ]:
# 医療診断の例でベイズの定理を理解
print("ベイズの定理：医療診断の例")
print("=" * 35)

# 設定
# - 病気の有病率: 0.1% (0.001)
# - 検査の感度: 99% (病気がある場合に陽性になる確率)
# - 検査の特異度: 95% (病気がない場合に陰性になる確率)

prior_disease = 0.001  # P(Disease)
sensitivity = 0.99     # P(Test+|Disease)
specificity = 0.95     # P(Test-|No Disease)

# P(Test+|No Disease) = 1 - specificity
false_positive_rate = 1 - specificity

# P(Test+) = P(Test+|Disease) * P(Disease) + P(Test+|No Disease) * P(No Disease)
prob_positive = (sensitivity * prior_disease + 
                false_positive_rate * (1 - prior_disease))

# ベイズの定理: P(Disease|Test+) = P(Test+|Disease) * P(Disease) / P(Test+)
posterior_disease = (sensitivity * prior_disease) / prob_positive

print(f"事前確率（有病率）: {prior_disease*100:.3f}%")
print(f"検査の感度: {sensitivity*100:.1f}%")
print(f"検査の特異度: {specificity*100:.1f}%")
print(f"\n検査陽性の確率: {prob_positive*100:.3f}%")
print(f"陽性時の病気の確率: {posterior_disease*100:.2f}%")
print(f"\n→ 検査が陽性でも、実際に病気の確率は{posterior_disease*100:.1f}%程度")

In [ ]:
# ベイズの定理の可視化
def bayes_analysis(prior_range, sensitivity, specificity):
    posteriors = []
    
    for prior in prior_range:
        false_pos_rate = 1 - specificity
        prob_pos = sensitivity * prior + false_pos_rate * (1 - prior)
        posterior = (sensitivity * prior) / prob_pos
        posteriors.append(posterior)
    
    return posteriors

# 有病率を変化させて事後確率を計算
prior_rates = np.logspace(-4, -1, 50)  # 0.01% to 10%
posteriors = bayes_analysis(prior_rates, sensitivity, specificity)

plt.figure(figsize=(12, 8))

# メインプロット
plt.subplot(2, 1, 1)
plt.semilogx(prior_rates * 100, np.array(posteriors) * 100, 'b-', linewidth=2)
plt.axvline(x=0.1, color='red', linestyle='--', alpha=0.7, label='元の例 (0.1%)')
plt.axhline(y=posterior_disease * 100, color='red', linestyle='--', alpha=0.7)
plt.xlabel('事前確率（有病率 %）')
plt.ylabel('事後確率（陽性時の病気確率 %）')
plt.title('ベイズの定理：事前確率と事後確率の関係')
plt.grid(True, alpha=0.3)
plt.legend()

# 比較プロット（直線スケール）
plt.subplot(2, 1, 2)
common_priors = np.linspace(0.001, 0.1, 100)
common_posteriors = bayes_analysis(common_priors, sensitivity, specificity)
plt.plot(common_priors * 100, np.array(common_posteriors) * 100, 'g-', linewidth=2)
plt.xlabel('事前確率（有病率 %）')
plt.ylabel('事後確率（陽性時の病気確率 %）')
plt.title('一般的な有病率範囲での事後確率')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 1.3 独立性と相関

In [ ]:
# 独立性の検証：2つのサイコロ
print("独立性の検証")
print("=" * 20)

# 2つのサイコロを10000回振る
n_trials = 10000
dice1 = np.random.randint(1, 7, n_trials)
dice2 = np.random.randint(1, 7, n_trials)

# 独立性の確認：P(A∩B) = P(A) × P(B)
# 事象A：サイコロ1が6
# 事象B：サイコロ2が6

prob_A = np.mean(dice1 == 6)
prob_B = np.mean(dice2 == 6)
prob_AB = np.mean((dice1 == 6) & (dice2 == 6))
prob_A_times_B = prob_A * prob_B

print(f"P(サイコロ1=6): {prob_A:.3f}")
print(f"P(サイコロ2=6): {prob_B:.3f}")
print(f"P(両方とも6): {prob_AB:.4f}")
print(f"P(A) × P(B): {prob_A_times_B:.4f}")
print(f"差: {abs(prob_AB - prob_A_times_B):.4f}")
print(f"→ {'独立' if abs(prob_AB - prob_A_times_B) < 0.01 else '従属'}")

In [ ]:
# 相関の例：身長と体重
print("\n相関の例：身長と体重")
print("=" * 25)

# 相関のあるデータを生成
n_people = 1000
heights = np.random.normal(170, 10, n_people)
# 体重は身長に相関がある（身長が高いほど重い傾向）
weights = 0.8 * heights + np.random.normal(-70, 8, n_people)

# 相関係数
correlation = np.corrcoef(heights, weights)[0, 1]
print(f"相関係数: {correlation:.3f}")

# 散布図
plt.figure(figsize=(10, 6))
plt.scatter(heights, weights, alpha=0.6, s=20)
plt.xlabel('身長 (cm)')
plt.ylabel('体重 (kg)')
plt.title(f'身長と体重の関係 (r = {correlation:.3f})')

# 回帰直線
z = np.polyfit(heights, weights, 1)
p = np.poly1d(z)
plt.plot(heights, p(heights), "r--", alpha=0.8, linewidth=2)
plt.grid(alpha=0.3)
plt.show()

print(f"回帰式: 体重 ≈ {z[0]:.2f} × 身長 + {z[1]:.1f}")

## 2. 離散確率分布の詳細

### 2.1 ベルヌーイ分布と二項分布

In [ ]:
# ベルヌーイ分布から二項分布への理解
print("ベルヌーイ分布から二項分布へ")
print("=" * 35)

# パラメータ
p = 0.3  # 成功確率
n = 20   # 試行回数

# 1. ベルヌーイ分布の繰り返しで二項分布をシミュレーション
n_simulations = 10000
binomial_results = []

for _ in range(n_simulations):
    # n回のベルヌーイ試行
    bernoulli_trials = np.random.binomial(1, p, n)
    success_count = np.sum(bernoulli_trials)
    binomial_results.append(success_count)

binomial_results = np.array(binomial_results)

# 2. 直接的な二項分布
binomial_direct = np.random.binomial(n, p, n_simulations)

# 結果の比較
print(f"ベルヌーイ試行の繰り返し - 平均: {np.mean(binomial_results):.2f}, 標準偏差: {np.std(binomial_results):.2f}")
print(f"直接的な二項分布 - 平均: {np.mean(binomial_direct):.2f}, 標準偏差: {np.std(binomial_direct):.2f}")
print(f"理論値 - 平均: {n*p:.2f}, 標準偏差: {np.sqrt(n*p*(1-p)):.2f}")

In [ ]:
# 二項分布の性質の可視化
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 異なるpでの二項分布
n_fixed = 20
p_values = [0.1, 0.3, 0.5, 0.7]
x = np.arange(0, n_fixed + 1)

for i, p_val in enumerate(p_values):
    row = i // 2
    col = i % 2
    
    # 理論値
    pmf = stats.binom.pmf(x, n_fixed, p_val)
    axes[row, col].bar(x, pmf, alpha=0.7, color=f'C{i}')
    
    # 統計量
    mean = n_fixed * p_val
    std = np.sqrt(n_fixed * p_val * (1 - p_val))
    
    axes[row, col].axvline(mean, color='red', linestyle='--', label=f'平均={mean:.1f}')
    axes[row, col].set_title(f'二項分布 B({n_fixed}, {p_val})')
    axes[row, col].set_xlabel('成功回数')
    axes[row, col].set_ylabel('確率')
    axes[row, col].legend()
    axes[row, col].grid(alpha=0.3)

plt.suptitle('異なるパラメータでの二項分布', fontsize=16)
plt.tight_layout()
plt.show()

### 2.2 ポアソン分布の深い理解

In [ ]:
# 二項分布からポアソン分布への極限
print("二項分布からポアソン分布への収束")
print("=" * 40)

# 固定されたλ = np
lambda_fixed = 5

# nを大きくしてpを小さくする
n_values = [10, 50, 100, 500, 1000]
x_max = 15
x = np.arange(0, x_max)

plt.figure(figsize=(15, 10))

# ポアソン分布（理論値）
poisson_pmf = stats.poisson.pmf(x, lambda_fixed)
plt.plot(x, poisson_pmf, 'ko-', linewidth=3, markersize=8, label=f'ポアソン分布 (λ={lambda_fixed})')

# 異なるnでの二項分布
colors = ['blue', 'green', 'orange', 'red', 'purple']
for i, n in enumerate(n_values):
    p = lambda_fixed / n
    if p <= 1:  # pは確率なので1以下
        binomial_pmf = stats.binom.pmf(x, n, p)
        plt.plot(x, binomial_pmf, 'o-', color=colors[i], alpha=0.7, 
                label=f'二項分布 B({n}, {p:.4f})')

plt.xlabel('発生回数')
plt.ylabel('確率')
plt.title(f'二項分布のポアソン分布への収束 (λ = np = {lambda_fixed})')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# 数値的比較
print("\n数値的比較（x=5での確率）:")
poisson_prob_5 = stats.poisson.pmf(5, lambda_fixed)
print(f"ポアソン分布: {poisson_prob_5:.6f}")

for n in [100, 500, 1000]:
    p = lambda_fixed / n
    binomial_prob_5 = stats.binom.pmf(5, n, p)
    error = abs(binomial_prob_5 - poisson_prob_5)
    print(f"二項分布 B({n}, {p:.4f}): {binomial_prob_5:.6f} (誤差: {error:.6f})")

In [ ]:
# ポアソン過程の実用例：ウェブサイトアクセス分析
print("ポアソン過程の実用例：ウェブサイトアクセス")
print("=" * 45)

# 1分あたり平均3アクセス
lambda_access = 3

# 24時間（1440分）のアクセスログをシミュレーション
access_counts = np.random.poisson(lambda_access, 1440)

# 時間帯別集計（1時間=60分ごと）
hourly_access = [np.sum(access_counts[i*60:(i+1)*60]) for i in range(24)]
hours = np.arange(24)

print(f"総アクセス数: {np.sum(access_counts)}")
print(f"平均アクセス数/分: {np.mean(access_counts):.2f}")
print(f"理論値: {lambda_access}/分")

# 可視化
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1分ごとのアクセス分布
axes[0, 0].hist(access_counts, bins=range(0, 12), density=True, alpha=0.7, 
                edgecolor='black', label='実測')
x_theory = np.arange(0, 12)
y_theory = stats.poisson.pmf(x_theory, lambda_access)
axes[0, 0].plot(x_theory, y_theory, 'ro-', label='理論値')
axes[0, 0].set_xlabel('1分あたりアクセス数')
axes[0, 0].set_ylabel('確率密度')
axes[0, 0].set_title('1分あたりのアクセス分布')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# 時間別アクセス数
axes[0, 1].bar(hours, hourly_access, alpha=0.7, color='green')
axes[0, 1].axhline(lambda_access * 60, color='red', linestyle='--', 
                   label=f'期待値 ({lambda_access * 60})')
axes[0, 1].set_xlabel('時間')
axes[0, 1].set_ylabel('1時間あたりアクセス数')
axes[0, 1].set_title('時間別アクセス数')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# 累積アクセス数
cumulative_access = np.cumsum(access_counts)
minutes = np.arange(1, 1441)
axes[1, 0].plot(minutes, cumulative_access, 'b-', alpha=0.7)
axes[1, 0].plot(minutes, lambda_access * minutes, 'r--', 
                label=f'理論値 ({lambda_access}t)')
axes[1, 0].set_xlabel('時間（分）')
axes[1, 0].set_ylabel('累積アクセス数')
axes[1, 0].set_title('累積アクセス数の推移')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# アクセス間隔の分布（指数分布）
# ポアソン過程では到着間隔は指数分布に従う
access_times = []
current_time = 0
for count in access_counts:
    if count > 0:
        intervals = np.random.exponential(1/lambda_access, count)
        for interval in intervals:
            current_time += interval
            access_times.append(current_time)

intervals_between_access = np.diff([0] + access_times[:1000])  # 最初の1000個
axes[1, 1].hist(intervals_between_access, bins=30, density=True, alpha=0.7, 
                edgecolor='black', label='実測')
x_exp = np.linspace(0, 2, 100)
y_exp = stats.expon.pdf(x_exp, scale=1/lambda_access)
axes[1, 1].plot(x_exp, y_exp, 'r-', linewidth=2, label='指数分布')
axes[1, 1].set_xlabel('アクセス間隔（分）')
axes[1, 1].set_ylabel('確率密度')
axes[1, 1].set_title('アクセス間隔の分布')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 2.3 幾何分布と負の二項分布

In [ ]:
# 幾何分布：最初の成功までの試行回数
print("幾何分布の例：くじ引き")
print("=" * 25)

# 当選確率1%のくじ
p_win = 0.01

# 幾何分布のシミュレーション
n_simulations = 10000
first_wins = []

for _ in range(n_simulations):
    trial = 1
    while np.random.random() > p_win:
        trial += 1
    first_wins.append(trial)

first_wins = np.array(first_wins)

print(f"最初の当選までの平均試行回数: {np.mean(first_wins):.1f}")
print(f"理論値: {1/p_win:.1f}")
print(f"標準偏差: {np.std(first_wins):.1f}")
print(f"理論値: {np.sqrt((1-p_win)/p_win**2):.1f}")

# 可視化
plt.figure(figsize=(12, 8))

# ヒストグラム
plt.subplot(2, 1, 1)
plt.hist(first_wins, bins=50, density=True, alpha=0.7, edgecolor='black', label='シミュレーション')

# 理論値
x_geom = np.arange(1, 300)
y_geom = stats.geom.pmf(x_geom, p_win)
plt.plot(x_geom, y_geom, 'r-', alpha=0.8, label='幾何分布')

plt.xlabel('最初の成功までの試行回数')
plt.ylabel('確率密度')
plt.title(f'幾何分布 (p = {p_win})')
plt.legend()
plt.grid(alpha=0.3)
plt.xlim(0, 500)

# 累積分布
plt.subplot(2, 1, 2)
x_range = np.arange(1, 500)
empirical_cdf = [np.mean(first_wins <= x) for x in x_range]
theoretical_cdf = stats.geom.cdf(x_range, p_win)

plt.plot(x_range, empirical_cdf, 'b-', alpha=0.7, label='経験分布')
plt.plot(x_range, theoretical_cdf, 'r--', label='理論分布')
plt.xlabel('試行回数')
plt.ylabel('累積確率')
plt.title('累積分布関数の比較')
plt.legend()
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 3. 連続確率分布の詳細

### 3.1 正規分布の深い性質

In [ ]:
# 中心極限定理の実演
print("中心極限定理の実演")
print("=" * 25)

# 非正規分布からのサンプル平均の分布
# 指数分布（非対称）を使用
original_dist = stats.expon(scale=2)

# 異なるサンプルサイズでのサンプル平均の分布
sample_sizes = [1, 5, 10, 30]
n_experiments = 5000

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

for i, n in enumerate(sample_sizes):
    # n個のサンプルの平均をn_experiments回計算
    sample_means = []
    for _ in range(n_experiments):
        samples = original_dist.rvs(n)
        sample_means.append(np.mean(samples))
    
    sample_means = np.array(sample_means)
    
    # ヒストグラム
    axes[i].hist(sample_means, bins=50, density=True, alpha=0.7, 
                 edgecolor='black', label=f'サンプル平均 (n={n})')
    
    # 理論的正規分布
    theoretical_mean = original_dist.mean()
    theoretical_std = original_dist.std() / np.sqrt(n)
    
    x = np.linspace(sample_means.min(), sample_means.max(), 100)
    y = stats.norm.pdf(x, theoretical_mean, theoretical_std)
    axes[i].plot(x, y, 'r-', linewidth=2, label='理論的正規分布')
    
    axes[i].set_title(f'サンプルサイズ n = {n}')
    axes[i].set_xlabel('サンプル平均')
    axes[i].set_ylabel('確率密度')
    axes[i].legend()
    axes[i].grid(alpha=0.3)
    
    # 統計情報
    print(f"n={n}: 平均={np.mean(sample_means):.3f}, 標準偏差={np.std(sample_means):.3f}")
    print(f"    理論値: 平均={theoretical_mean:.3f}, 標準偏差={theoretical_std:.3f}")

plt.suptitle('中心極限定理：指数分布からのサンプル平均の分布', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# 正規分布の線形結合
print("\n正規分布の線形結合")
print("=" * 25)

# X ~ N(μ₁, σ₁²), Y ~ N(μ₂, σ₂²)のとき
# aX + bY + c ~ N(aμ₁ + bμ₂ + c, a²σ₁² + b²σ₂²) (独立の場合)

mu1, sigma1 = 10, 3
mu2, sigma2 = 20, 4
a, b, c = 2, -1, 5

# シミュレーション
n_samples = 10000
X = np.random.normal(mu1, sigma1, n_samples)
Y = np.random.normal(mu2, sigma2, n_samples)
Z = a * X + b * Y + c

# 理論値
theoretical_mean = a * mu1 + b * mu2 + c
theoretical_var = a**2 * sigma1**2 + b**2 * sigma2**2
theoretical_std = np.sqrt(theoretical_var)

print(f"Z = {a}X + ({b})Y + {c}")
print(f"実測: 平均={np.mean(Z):.3f}, 標準偏差={np.std(Z):.3f}")
print(f"理論: 平均={theoretical_mean:.3f}, 標準偏差={theoretical_std:.3f}")

# 可視化
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(X, bins=50, density=True, alpha=0.7, label=f'X ~ N({mu1}, {sigma1}²)')
x_range = np.linspace(X.min(), X.max(), 100)
plt.plot(x_range, stats.norm.pdf(x_range, mu1, sigma1), 'r-', linewidth=2)
plt.title('分布 X')
plt.legend()
plt.grid(alpha=0.3)

plt.subplot(1, 3, 2)
plt.hist(Y, bins=50, density=True, alpha=0.7, label=f'Y ~ N({mu2}, {sigma2}²)')
y_range = np.linspace(Y.min(), Y.max(), 100)
plt.plot(y_range, stats.norm.pdf(y_range, mu2, sigma2), 'r-', linewidth=2)
plt.title('分布 Y')
plt.legend()
plt.grid(alpha=0.3)

plt.subplot(1, 3, 3)
plt.hist(Z, bins=50, density=True, alpha=0.7, label=f'Z = {a}X + ({b})Y + {c}')
z_range = np.linspace(Z.min(), Z.max(), 100)
plt.plot(z_range, stats.norm.pdf(z_range, theoretical_mean, theoretical_std), 'r-', linewidth=2)
plt.title('線形結合 Z')
plt.legend()
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 3.2 その他の重要な連続分布

In [ ]:
# χ²分布、t分布、F分布の関係
print("統計的推定に重要な分布")
print("=" * 30)

# パラメータ設定
df_values = [1, 3, 5, 10, 30]
n_samples = 5000

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# χ²分布
x_chi2 = np.linspace(0, 30, 200)
for df in df_values:
    y_chi2 = stats.chi2.pdf(x_chi2, df)
    axes[0, 0].plot(x_chi2, y_chi2, label=f'df={df}')

axes[0, 0].set_title('χ²分布')
axes[0, 0].set_xlabel('x')
axes[0, 0].set_ylabel('確率密度')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# t分布と正規分布の比較
x_t = np.linspace(-4, 4, 200)
# 正規分布
y_norm = stats.norm.pdf(x_t, 0, 1)
axes[0, 1].plot(x_t, y_norm, 'k-', linewidth=2, label='標準正規分布')

for df in [1, 3, 10, 30]:
    y_t = stats.t.pdf(x_t, df)
    axes[0, 1].plot(x_t, y_t, label=f't分布 (df={df})')

axes[0, 1].set_title('t分布の正規分布への収束')
axes[0, 1].set_xlabel('x')
axes[0, 1].set_ylabel('確率密度')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# F分布
x_f = np.linspace(0, 5, 200)
df_combinations = [(1, 1), (5, 5), (10, 10), (30, 30)]
for df1, df2 in df_combinations:
    y_f = stats.f.pdf(x_f, df1, df2)
    axes[1, 0].plot(x_f, y_f, label=f'F({df1},{df2})')

axes[1, 0].set_title('F分布')
axes[1, 0].set_xlabel('x')
axes[1, 0].set_ylabel('確率密度')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# ベータ分布（0-1に制限された分布）
x_beta = np.linspace(0, 1, 200)
beta_params = [(0.5, 0.5), (1, 1), (2, 2), (5, 2), (2, 5)]
for alpha, beta in beta_params:
    y_beta = stats.beta.pdf(x_beta, alpha, beta)
    axes[1, 1].plot(x_beta, y_beta, label=f'β({alpha},{beta})')

axes[1, 1].set_title('ベータ分布')
axes[1, 1].set_xlabel('x')
axes[1, 1].set_ylabel('確率密度')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 4. 実践的応用例

### 4.1 A/Bテストの統計的設計

In [ ]:
# A/Bテストのサンプルサイズ設計
print("A/Bテストのサンプルサイズ設計")
print("=" * 35)

# パラメータ設定
p1 = 0.05      # コントロール群のコンバージョン率
p2 = 0.06      # 治療群のコンバージョン率（期待改善後）
alpha = 0.05   # 有意水準（タイプIエラー率）
beta = 0.20    # タイプIIエラー率（検出力 = 1-β = 0.8）

# 効果量（Cohen's h）
effect_size = 2 * (np.arcsin(np.sqrt(p2)) - np.arcsin(np.sqrt(p1)))
print(f"効果量 (Cohen's h): {effect_size:.4f}")

# 必要サンプルサイズの概算（等分散の場合）
z_alpha = stats.norm.ppf(1 - alpha/2)  # 両側検定
z_beta = stats.norm.ppf(1 - beta)

# 各群のサンプルサイズ
pooled_p = (p1 + p2) / 2
n_per_group = ((z_alpha * np.sqrt(2 * pooled_p * (1 - pooled_p)) + 
                z_beta * np.sqrt(p1 * (1 - p1) + p2 * (1 - p2))) / (p2 - p1))**2

n_per_group = int(np.ceil(n_per_group))
total_sample = n_per_group * 2

print(f"\n必要サンプルサイズ:")
print(f"各群: {n_per_group}人")
print(f"総計: {total_sample}人")

# シミュレーションによる検証
n_simulations = 1000
significant_results = 0

for _ in range(n_simulations):
    # データ生成
    control_conversions = np.random.binomial(n_per_group, p1)
    treatment_conversions = np.random.binomial(n_per_group, p2)
    
    # 比率の差の検定（Z検定の近似）
    p1_hat = control_conversions / n_per_group
    p2_hat = treatment_conversions / n_per_group
    
    pooled_p_hat = (control_conversions + treatment_conversions) / (2 * n_per_group)
    se = np.sqrt(pooled_p_hat * (1 - pooled_p_hat) * (1/n_per_group + 1/n_per_group))
    
    if se > 0:
        z_stat = (p2_hat - p1_hat) / se
        p_value = 2 * (1 - stats.norm.cdf(abs(z_stat)))
        
        if p_value < alpha:
            significant_results += 1

empirical_power = significant_results / n_simulations
print(f"\nシミュレーション結果:")
print(f"検出力（実測）: {empirical_power:.3f}")
print(f"検出力（目標）: {1-beta:.3f}")

In [ ]:
# A/Bテストの結果の可視化
# 実際のテスト結果をシミュレーション
np.random.seed(42)

# より大きなサンプルでテスト実行
n_actual = 5000
control_results = np.random.binomial(1, p1, n_actual)
treatment_results = np.random.binomial(1, p2, n_actual)

# 累積コンバージョン率の推移
control_cumulative = np.cumsum(control_results) / np.arange(1, n_actual + 1)
treatment_cumulative = np.cumsum(treatment_results) / np.arange(1, n_actual + 1)

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 累積コンバージョン率
x_axis = np.arange(1, n_actual + 1)
axes[0, 0].plot(x_axis, control_cumulative, 'b-', alpha=0.7, label='コントロール群')
axes[0, 0].plot(x_axis, treatment_cumulative, 'r-', alpha=0.7, label='治療群')
axes[0, 0].axhline(p1, color='blue', linestyle='--', alpha=0.5, label=f'真の値 (Control: {p1})')
axes[0, 0].axhline(p2, color='red', linestyle='--', alpha=0.5, label=f'真の値 (Treatment: {p2})')
axes[0, 0].set_xlabel('サンプル数')
axes[0, 0].set_ylabel('累積コンバージョン率')
axes[0, 0].set_title('コンバージョン率の収束')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# 信頼区間の推移
sample_points = np.arange(100, n_actual, 100)
control_ci_lower = []
control_ci_upper = []
treatment_ci_lower = []
treatment_ci_upper = []

for n in sample_points:
    # コントロール群の信頼区間
    p_hat = np.sum(control_results[:n]) / n
    se = np.sqrt(p_hat * (1 - p_hat) / n)
    margin = 1.96 * se
    control_ci_lower.append(p_hat - margin)
    control_ci_upper.append(p_hat + margin)
    
    # 治療群の信頼区間
    p_hat = np.sum(treatment_results[:n]) / n
    se = np.sqrt(p_hat * (1 - p_hat) / n)
    margin = 1.96 * se
    treatment_ci_lower.append(p_hat - margin)
    treatment_ci_upper.append(p_hat + margin)

axes[0, 1].fill_between(sample_points, control_ci_lower, control_ci_upper, 
                        alpha=0.3, color='blue', label='コントロール群 95%CI')
axes[0, 1].fill_between(sample_points, treatment_ci_lower, treatment_ci_upper, 
                        alpha=0.3, color='red', label='治療群 95%CI')
axes[0, 1].plot(sample_points, [control_cumulative[n-1] for n in sample_points], 'b-', alpha=0.7)
axes[0, 1].plot(sample_points, [treatment_cumulative[n-1] for n in sample_points], 'r-', alpha=0.7)
axes[0, 1].set_xlabel('サンプル数')
axes[0, 1].set_ylabel('コンバージョン率')
axes[0, 1].set_title('信頼区間の推移')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# 最終結果の分布
final_control_rate = np.mean(control_results)
final_treatment_rate = np.mean(treatment_results)

# ブートストラップによる分布推定
n_bootstrap = 1000
control_bootstrap = [np.mean(np.random.choice(control_results, n_actual, replace=True)) 
                    for _ in range(n_bootstrap)]
treatment_bootstrap = [np.mean(np.random.choice(treatment_results, n_actual, replace=True)) 
                      for _ in range(n_bootstrap)]

axes[1, 0].hist(control_bootstrap, bins=30, alpha=0.7, color='blue', 
               label=f'コントロール群 ({final_control_rate:.3f})')
axes[1, 0].hist(treatment_bootstrap, bins=30, alpha=0.7, color='red', 
               label=f'治療群 ({final_treatment_rate:.3f})')
axes[1, 0].set_xlabel('コンバージョン率')
axes[1, 0].set_ylabel('頻度')
axes[1, 0].set_title('ブートストラップ分布')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# 差の分布
difference_bootstrap = np.array(treatment_bootstrap) - np.array(control_bootstrap)
axes[1, 1].hist(difference_bootstrap, bins=30, alpha=0.7, color='green')
axes[1, 1].axvline(0, color='red', linestyle='--', label='差=0')
axes[1, 1].axvline(np.mean(difference_bootstrap), color='black', linestyle='-', 
                  label=f'平均差={np.mean(difference_bootstrap):.4f}')
axes[1, 1].set_xlabel('コンバージョン率の差 (Treatment - Control)')
axes[1, 1].set_ylabel('頻度')
axes[1, 1].set_title('効果サイズの分布')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# 統計的検定の結果
from scipy.stats import chi2_contingency

# 2×2分割表
control_conv = np.sum(control_results)
control_total = len(control_results)
treatment_conv = np.sum(treatment_results)
treatment_total = len(treatment_results)

contingency_table = np.array([
    [control_conv, control_total - control_conv],
    [treatment_conv, treatment_total - treatment_conv]
])

chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

print(f"\n最終的なA/Bテスト結果:")
print(f"コントロール群: {control_conv}/{control_total} ({final_control_rate:.3f})")
print(f"治療群: {treatment_conv}/{treatment_total} ({final_treatment_rate:.3f})")
print(f"差: {final_treatment_rate - final_control_rate:.4f}")
print(f"相対的改善: {(final_treatment_rate - final_control_rate)/final_control_rate*100:.1f}%")
print(f"\nカイ二乗検定:")
print(f"統計量: {chi2_stat:.4f}")
print(f"p値: {p_value:.6f}")
print(f"結果: {'有意' if p_value < 0.05 else '有意でない'} (α=0.05)")

### 4.2 リスク管理：VaR（Value at Risk）の詳細計算

In [ ]:
# VaRの異なる計算方法の比較
print("VaRの計算方法比較")
print("=" * 25)

# 株価リターンのシミュレーション（実際はより複雑な分布）
np.random.seed(42)
n_days = 1000

# 正規分布仮定（簡化されたモデル）
daily_return_mean = 0.0005  # 年率約12%の期待リターン
daily_return_std = 0.02     # 年率約30%のボラティリティ

returns_normal = np.random.normal(daily_return_mean, daily_return_std, n_days)

# より現実的なt分布（ファットテール）
df_t = 5  # 自由度
# t分布を標準化して調整
returns_t = stats.t.rvs(df_t, size=n_days) * daily_return_std / np.sqrt(df_t/(df_t-2)) + daily_return_mean

# 信頼水準
confidence_levels = [0.95, 0.99, 0.999]

print("VaR計算結果（1日、投資額1億円）:")
print("=" * 40)

investment_amount = 100_000_000  # 1億円

for confidence in confidence_levels:
    alpha = 1 - confidence
    
    # 1. パラメトリック法（正規分布仮定）
    var_parametric = -stats.norm.ppf(alpha, daily_return_mean, daily_return_std) * investment_amount
    
    # 2. ヒストリカル法（正規分布データ）
    var_historical_normal = -np.percentile(returns_normal, alpha * 100) * investment_amount
    
    # 3. ヒストリカル法（t分布データ）
    var_historical_t = -np.percentile(returns_t, alpha * 100) * investment_amount
    
    # 4. モンテカルロ法
    mc_returns = np.random.normal(daily_return_mean, daily_return_std, 100000)
    var_monte_carlo = -np.percentile(mc_returns, alpha * 100) * investment_amount
    
    print(f"\n信頼水準 {confidence*100:.1f}% ({alpha*100:.1f}パーセンタイル):")
    print(f"  パラメトリック法: {var_parametric/1_000_000:.1f}百万円")
    print(f"  ヒストリカル法(正規): {var_historical_normal/1_000_000:.1f}百万円")
    print(f"  ヒストリカル法(t分布): {var_historical_t/1_000_000:.1f}百万円")
    print(f"  モンテカルロ法: {var_monte_carlo/1_000_000:.1f}百万円")

In [ ]:
# VaRの視覚的比較
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# リターン分布の比較
axes[0, 0].hist(returns_normal, bins=50, density=True, alpha=0.7, 
               color='blue', label='正規分布', edgecolor='black')
axes[0, 0].hist(returns_t, bins=50, density=True, alpha=0.7, 
               color='red', label='t分布', edgecolor='black')

# VaRラインを表示
var_95_normal = np.percentile(returns_normal, 5)
var_95_t = np.percentile(returns_t, 5)
axes[0, 0].axvline(var_95_normal, color='blue', linestyle='--', alpha=0.8, label='95% VaR (正規)')
axes[0, 0].axvline(var_95_t, color='red', linestyle='--', alpha=0.8, label='95% VaR (t分布)')

axes[0, 0].set_xlabel('日次リターン')
axes[0, 0].set_ylabel('確率密度')
axes[0, 0].set_title('リターン分布の比較')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Q-Qプロット（分布の比較）
from scipy.stats import probplot
probplot(returns_normal, dist="norm", plot=axes[0, 1])
axes[0, 1].set_title('正規分布のQ-Qプロット')
axes[0, 1].grid(alpha=0.3)

# 累積損失の推移
cumulative_returns_normal = np.cumsum(returns_normal)
cumulative_returns_t = np.cumsum(returns_t)

axes[1, 0].plot(cumulative_returns_normal, color='blue', alpha=0.7, label='正規分布')
axes[1, 0].plot(cumulative_returns_t, color='red', alpha=0.7, label='t分布')
axes[1, 0].set_xlabel('日数')
axes[1, 0].set_ylabel('累積リターン')
axes[1, 0].set_title('累積リターンの推移')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# VaRのバックテスト
var_95_threshold = var_95_normal
violations_normal = returns_normal < var_95_threshold
violation_rate_normal = np.mean(violations_normal)

var_95_threshold_t = var_95_t  
violations_t = returns_t < var_95_threshold_t
violation_rate_t = np.mean(violations_t)

# 実際の損失 vs VaR
days = np.arange(len(returns_normal))
axes[1, 1].scatter(days[violations_normal], returns_normal[violations_normal] * investment_amount / 1_000_000, 
                  color='red', alpha=0.6, s=20, label=f'VaR違反 (正規): {violation_rate_normal:.1%}')
axes[1, 1].scatter(days[~violations_normal], returns_normal[~violations_normal] * investment_amount / 1_000_000, 
                  color='blue', alpha=0.3, s=10, label='通常')
axes[1, 1].axhline(-var_parametric/1_000_000, color='black', linestyle='--', 
                  label=f'95% VaR: {var_parametric/1_000_000:.1f}百万円')
axes[1, 1].set_xlabel('日数')
axes[1, 1].set_ylabel('損益 (百万円)')
axes[1, 1].set_title('VaRバックテスト')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nバックテスト結果:")
print(f"期待違反率: 5.0%")
print(f"実際の違反率（正規分布）: {violation_rate_normal:.1%}")
print(f"実際の違反率（t分布）: {violation_rate_t:.1%}")
print(f"\n→ t分布の方が現実的（ファットテールを考慮）")

## 練習問題

### 問題1: ベイズ統計の応用
迷惑メールフィルターの問題です。

In [ ]:
# 迷惑メールフィルターの設定
# - 全メールの30%が迷惑メール
# - 迷惑メールの90%で「無料」というキーワードが含まれる
# - 正常メールの5%で「無料」というキーワードが含まれる

prior_spam = 0.30
likelihood_free_given_spam = 0.90
likelihood_free_given_ham = 0.05

# ここに解答を書いてください
# 1. 「無料」というキーワードを含むメールが迷惑メールである確率を計算（ベイズの定理）

# 2. シミュレーションで1000通のメールを生成し、上記の確率を検証

# 3. 異なる事前確率（10%, 50%, 70%）での事後確率の変化を可視化


### 問題2: 中心極限定理の実験
指数分布からのサンプリング実験です。

In [ ]:
# 指数分布（λ=2）からのサンプリング
lambda_param = 2

# ここに解答を書いてください
# 1. 異なるサンプルサイズ（n=1,5,10,30,100）でのサンプル平均の分布を作成

# 2. 各サンプルサイズでの理論的平均と標準偏差を計算

# 3. シミュレーション結果と理論値を比較する可視化を作成

# 4. 正規性の検定（Shapiro-Wilk検定）を各サンプルサイズで実行


### 問題3: ポートフォリオ最適化の基礎
2つの資産からなるポートフォリオのリスク分析です。

In [ ]:
# 2つの資産のリターン特性
mu1, sigma1 = 0.08, 0.15  # 資産1: 期待リターン8%, リスク15%
mu2, sigma2 = 0.12, 0.25  # 資産2: 期待リターン12%, リスク25%
correlation = 0.3         # 相関係数

# ここに解答を書いてください
# 1. 異なる投資比率（0%, 25%, 50%, 75%, 100%）での
#    ポートフォリオの期待リターンとリスクを計算

# 2. 効率フロンティア（リスク-リターン曲線）を描画

# 3. シャープレシオが最大となる最適ポートフォリオを見つける
#    （無リスク資産の利率を3%と仮定）

# 4. 相関係数を変化させたときの効率フロンティアの変化を可視化


## まとめ

今日学んだ確率・分布の概念：

1. **確率論の基礎**
   - コルモゴロフの公理
   - 条件付き確率とベイズの定理
   - 独立性と相関

2. **離散確率分布**
   - ベルヌーイ分布・二項分布
   - ポアソン分布とポアソン過程
   - 幾何分布・負の二項分布

3. **連続確率分布**
   - 正規分布と中心極限定理
   - χ²分布、t分布、F分布
   - ベータ分布、指数分布

4. **実践応用**
   - A/Bテストの統計的設計
   - VaRによるリスク管理
   - ベイズ統計の実用例

これらの確率分布は、機械学習のアルゴリズム、
統計的推論、リスク管理など、
データサイエンスのあらゆる分野で活用されます。